# Lattice test

In [1]:
from orbit.lattice import AccLattice, AccNode, AccActionsContainer

In [2]:
import sys
import math
import posix

In [3]:
def Blanks(n):
    s = ""
    for i in range(n):
        s += " "
    return s

def funcEntrance(paramsDict):
    nLevel[0] += 1
    node = paramsDict["node"]
    if "print" in paramsDict and paramsDict["print"] == True:
        print(Blanks(nLevel[0]), "ENTER level=", nLevel[0], " node=", node.getName())
        nElems[0] += 1


def funcExit(paramsDict):
    node = paramsDict["node"]
    if "print" in paramsDict and paramsDict["print"] == True:
        print(Blanks(nLevel[0]), "EXIT  level=", nLevel[0], " node=", node.getName())
    nLevel[0] -= 1


def funcTrack(paramsDict):
    node = paramsDict["node"]
    if "print" in paramsDict and paramsDict["print"] == True:
        print(Blanks(nLevel[0]), "BODY TRACK through node =", node.getName(), " level=", nLevel[0])



In [4]:
nLevel = [0]
nElems = [0]


lattice = AccLattice("test_lattice")

elem1 = AccNode("el-1")
elem2 = AccNode("el-2")
elem3 = AccNode("el-3")

elem1.setLength(1.1)
elem2.setLength(2.1)
elem3.setLength(3.1)

lattice.addNode(elem1)
lattice.addNode(elem2)
lattice.addNode(elem3)

elem1_1 = AccNode("el-1-1")
elem1_1.setnParts(2)
elem1_1_1 = AccNode("el-1-1-1")
elem1_1_2 = AccNode("el-1-1-2")
elem1_1_3 = AccNode("el-1-1-3")
elem1_1_4 = AccNode("el-1-1-4")

elem1.addChildNode(elem1_1, AccNode.ENTRANCE)
elem1_1.addChildNode(elem1_1_1, AccNode.ENTRANCE)
elem1_1.addChildNode(elem1_1_2, AccNode.BODY, 0)
elem1_1.addChildNode(elem1_1_3, AccNode.BODY, 1)
elem1_1.addChildNode(elem1_1_4, AccNode.EXIT)


elem1_2 = AccNode("el-1-2")
elem2.addChildNode(elem1_2, AccNode.EXIT)

acts = AccActionsContainer()


acts.addAction(funcEntrance, AccActionsContainer.ENTRANCE)
acts.addAction(funcTrack, AccActionsContainer.BODY)
acts.addAction(funcExit, AccActionsContainer.EXIT)

lattice.initialize()

In [5]:
print("Total length=", lattice.getLength())

Total length= 6.300000000000001


In [6]:
nodes = lattice.getNodes()
for node in nodes:
    print("node=", node.getName(), " s start,stop = %4.3f %4.3f " % lattice.getNodePositionsDict()[node])

node= el-1  s start,stop = 0.000 1.100 
node= el-2  s start,stop = 1.100 3.200 
node= el-3  s start,stop = 3.200 6.300 


In [7]:
d = {"print": True}

In [8]:
lattice.trackActions(acts, d)

  ENTER level= 1  node= el-1
   ENTER level= 2  node= el-1-1
    ENTER level= 3  node= el-1-1-1
    BODY TRACK through node = el-1-1-1  level= 3
    EXIT  level= 3  node= el-1-1-1
    ENTER level= 3  node= el-1-1-2
    BODY TRACK through node = el-1-1-2  level= 3
    EXIT  level= 3  node= el-1-1-2
   BODY TRACK through node = el-1-1  level= 2
    ENTER level= 3  node= el-1-1-3
    BODY TRACK through node = el-1-1-3  level= 3
    EXIT  level= 3  node= el-1-1-3
   BODY TRACK through node = el-1-1  level= 2
    ENTER level= 3  node= el-1-1-4
    BODY TRACK through node = el-1-1-4  level= 3
    EXIT  level= 3  node= el-1-1-4
   EXIT  level= 2  node= el-1-1
  BODY TRACK through node = el-1  level= 1
  EXIT  level= 1  node= el-1
  ENTER level= 1  node= el-2
  BODY TRACK through node = el-2  level= 1
   ENTER level= 2  node= el-1-2
   BODY TRACK through node = el-1-2  level= 2
   EXIT  level= 2  node= el-1-2
  EXIT  level= 1  node= el-2
  ENTER level= 1  node= el-3
  BODY TRACK through node =

In [9]:
print("Total number of nodes=", nElems[0])
# ========Speed test==========================
count = 1
while count < 100000:
    # lattice.initialize()
    lattice.trackActions(acts)
    if count % 10000 == 0:
        print("i=", count, " time= %9.8f " % (posix.times()[0] / count))
    count += 1

print("====STOP===")

#sys.exit(0)

Total number of nodes= 9
i= 10000  time= 0.00005000 
i= 20000  time= 0.00003150 
i= 30000  time= 0.00002533 
i= 40000  time= 0.00002225 
i= 50000  time= 0.00002040 
i= 60000  time= 0.00001917 
i= 70000  time= 0.00001814 
i= 80000  time= 0.00001750 
i= 90000  time= 0.00001689 
====STOP===


# bunch generator tests

In [10]:
#!/usr/bin/env python

# --------------------------------------------------------
# The script will test the gerenrators from bunch_generators
# --------------------------------------------------------

import random
import pytest

In [11]:
random.seed(100)
from orbit.bunch_generators import TwissContainer, TwissAnalysis
from orbit.bunch_generators import WaterBagDist3D, GaussDist3D, KVDist3D

In [12]:
n = 10000
# ---------------------------------------------
# KV 3D
# ---------------------------------------------
twissX = TwissContainer(alpha=1.0, beta=2.0, emittance=3.0)
twissY = TwissContainer(alpha=2.0, beta=3.0, emittance=4.0)
twissZ = TwissContainer(alpha=3.0, beta=4.0, emittance=5.0)
dist = KVDist3D(twissX, twissY, twissZ)
twiss_analysis = TwissAnalysis(3)
for i in range(n):
    (x, xp, y, yp, z, zp) = dist.getCoordinates()
    twiss_analysis.account((x, xp, y, yp, z, zp))

print("=================================================")
print("KV 3D - done!")
print("                  alpha       beta [m/rad]    gamma     emitt[m*rad] ")
print("Twiss     X  %12.5g  %12.5g   %12.5g    %12.5g " % twissX.getAlphaBetaGammaEmitt())
print("Generated X  %12.5g  %12.5g   %12.5g    %12.5g " % twiss_analysis.getTwiss(0))
print("......................................................................")
print("Twiss     Y  %12.5g  %12.5g   %12.5g    %12.5g " % twissY.getAlphaBetaGammaEmitt())
print("Generated Y  %12.5g  %12.5g   %12.5g    %12.5g " % twiss_analysis.getTwiss(1))
print("......................................................................")
print("Twiss     Z  %12.5g  %12.5g   %12.5g    %12.5g " % twissZ.getAlphaBetaGammaEmitt())
print("Generated Z  %12.5g  %12.5g   %12.5g    %12.5g " % twiss_analysis.getTwiss(2))
print("=================================================")

KV 3D - done!
                  alpha       beta [m/rad]    gamma     emitt[m*rad] 
Twiss     X             1             2              1               3 
Generated X       0.98779        1.9908        0.99243          2.9966 
......................................................................
Twiss     Y             2             3         1.6667               4 
Generated Y        2.0165        3.0098         1.6833          3.9872 
......................................................................
Twiss     Z             3             4            2.5               5 
Generated Z        3.0347        4.0329         2.5316          5.0201 


In [13]:
# pytest for KVDist3D
get_KV3D_Twiss_X = "%.5g %.5g %.5g %.5g" % twiss_analysis.getTwiss(0)

In [14]:
def test_kv3d_twissX_generator():
    assert get_KV3D_Twiss_X == "0.98779 1.9908 0.99243 2.9966"

In [15]:
get_KV3D_Twiss_Y = "%.5g %.5g %.5g %.5g" % twiss_analysis.getTwiss(1)

In [16]:
get_KV3D_Twiss_Y

'2.0165 3.0098 1.6833 3.9872'

In [17]:
def test_kv3d_twissY_generator():
    assert get_KV3D_Twiss_Y == "2.0165 3.0098 1.6833 3.9872"

In [18]:
get_KV3D_Twiss_Z = "%.5g %.5g %.5g %.5g" % twiss_analysis.getTwiss(2)

In [19]:
def test_kv3d_twissZ_generator():
    assert get_KV3D_Twiss_Z == "3.0347 4.0329 2.5316 5.0201"

In [20]:
# ---------------------------------------------
# Water Bag 3D
# ---------------------------------------------
dist = WaterBagDist3D(twissX, twissY, twissZ)
twiss_analysis.init()
for i in range(n):
    (x, xp, y, yp, z, zp) = dist.getCoordinates()
    twiss_analysis.account((x, xp, y, yp, z, zp))

print("=================================================")
print("Water Bag 3D - done!")
print("                  alpha       beta [m/rad]    gamma     emitt[m*rad] ")
print("Twiss     X  %12.5g  %12.5g   %12.5g    %12.5g " % twissX.getAlphaBetaGammaEmitt())
print("Generated X  %12.5g  %12.5g   %12.5g    %12.5g " % twiss_analysis.getTwiss(0))
print("......................................................................")
print("Twiss     Y  %12.5g  %12.5g   %12.5g    %12.5g " % twissY.getAlphaBetaGammaEmitt())
print("Generated Y  %12.5g  %12.5g   %12.5g    %12.5g " % twiss_analysis.getTwiss(1))
print("......................................................................")
print("Twiss     Z  %12.5g  %12.5g   %12.5g    %12.5g " % twissZ.getAlphaBetaGammaEmitt())
print("Generated Z  %12.5g  %12.5g   %12.5g    %12.5g " % twiss_analysis.getTwiss(2))
print("=================================================")

Water Bag 3D - done!
                  alpha       beta [m/rad]    gamma     emitt[m*rad] 
Twiss     X             1             2              1               3 
Generated X       0.98348        1.9663         1.0005          2.9886 
......................................................................
Twiss     Y             2             3         1.6667               4 
Generated Y         1.995        3.0085         1.6553          3.9596 
......................................................................
Twiss     Z             3             4            2.5               5 
Generated Z        3.0269        4.0174         2.5295          5.0614 


In [21]:
# Pytest for Water Bag 3D
get_WaterBag3D_Twiss_X = "%.5g %.5g %.5g %.5g" % twiss_analysis.getTwiss(0)

In [22]:
def test_waterbag3d_twissX_generator():
    assert get_WaterBag3D_Twiss_X == "0.98348 1.9663 1.0005 2.9886"

In [23]:
get_WaterBag3D_Twiss_Y = "%.5g %.5g %.5g %.5g" % twiss_analysis.getTwiss(1)

In [24]:
def test_waterbag3d_twissY_generator():
    assert get_WaterBag3D_Twiss_Y == "1.995 3.0085 1.6553 3.9596"

In [25]:
get_WaterBag3D_Twiss_Z = "%.5g %.5g %.5g %.5g" % twiss_analysis.getTwiss(2)

In [26]:
def test_waterbag3d_twissZ_generator():
    assert get_WaterBag3D_Twiss_Z == "3.0269 4.0174 2.5295 5.0614"

In [27]:
# ---------------------------------------------
# Gauss 3D
# ---------------------------------------------
dist = GaussDist3D(twissX, twissY, twissZ)
twiss_analysis.init()
for i in range(n):
    (x, xp, y, yp, z, zp) = dist.getCoordinates()
    twiss_analysis.account((x, xp, y, yp, z, zp))

print("=================================================")
print("Gauss 3D - done!")
print("                  alpha       beta [m/rad]    gamma     emitt[m*rad] ")
print("Twiss     X  %12.5g  %12.5g   %12.5g    %12.5g " % twissX.getAlphaBetaGammaEmitt())
print("Generated X  %12.5g  %12.5g   %12.5g    %12.5g " % twiss_analysis.getTwiss(0))
print("......................................................................")
print("Twiss     Y  %12.5g  %12.5g   %12.5g    %12.5g " % twissY.getAlphaBetaGammaEmitt())
print("Generated Y  %12.5g  %12.5g   %12.5g    %12.5g " % twiss_analysis.getTwiss(1))
print("......................................................................")
print("Twiss     Z  %12.5g  %12.5g   %12.5g    %12.5g " % twissZ.getAlphaBetaGammaEmitt())
print("Generated Z  %12.5g  %12.5g   %12.5g    %12.5g " % twiss_analysis.getTwiss(2))
print("=================================================")

Gauss 3D - done!
                  alpha       beta [m/rad]    gamma     emitt[m*rad] 
Twiss     X             1             2              1               3 
Generated X        1.0149        2.0058          1.012          3.0533 
......................................................................
Twiss     Y             2             3         1.6667               4 
Generated Y         2.011        3.0206         1.6699          4.0153 
......................................................................
Twiss     Z             3             4            2.5               5 
Generated Z        3.0053        3.9921         2.5129          5.0294 


In [28]:
# pytests for GaussDist3D
get_Gauss3D_Twiss_X = "%.5g %.5g %.5g %.5g" % twiss_analysis.getTwiss(0)

In [29]:
def test_gauss3d_twissX_generator():
    assert get_Gauss3D_Twiss_X == "1.0149 2.0058 1.012 3.0533"

In [30]:
test_gauss3d_twissX_generator()

In [31]:
get_Gauss3D_Twiss_Y = "%.5g %.5g %.5g %.5g" % twiss_analysis.getTwiss(1)

In [32]:
def test_gauss3d_twissY_generator():
    assert get_Gauss3D_Twiss_Y == "2.011 3.0206 1.6699 4.0153"

In [33]:
get_Gauss3D_Twiss_Z = "%.5g %.5g %.5g %.5g" % twiss_analysis.getTwiss(2)

In [34]:
def test_gauss3d_twissZ_generator():
    assert get_Gauss3D_Twiss_Z == "3.0053 3.9921 2.5129 5.0294"

# Space Charge 2.5D test

In [35]:
# -----------------------------------------------------
# Creates Space Charge Calculator Node
# -----------------------------------------------------
import math
import random

In [36]:
random.seed(10)

In [37]:
from orbit.teapot import teapot
from orbit.space_charge.sc2p5d import scLatticeModifications
from orbit.core.bunch import Bunch
from orbit.core.spacecharge import SpaceChargeCalc2p5D, Boundary2D

/home/hr/Codes/pyorbit3/PyORBIT3/py/orbit/parsers/mad_parser.py:543: SyntaxWarning: invalid escape sequence '\.'
  str_out = re.sub("\.e", ".0e", str_in)
/home/hr/Codes/pyorbit3/PyORBIT3/py/orbit/parsers/mad_parser.py:545: SyntaxWarning: invalid escape sequence '\('
  str_out = re.sub("sin\(", "math.sin(", str_out)
/home/hr/Codes/pyorbit3/PyORBIT3/py/orbit/parsers/mad_parser.py:546: SyntaxWarning: invalid escape sequence '\('
  str_out = re.sub("SIN\(", "math.sin(", str_out)
/home/hr/Codes/pyorbit3/PyORBIT3/py/orbit/parsers/mad_parser.py:547: SyntaxWarning: invalid escape sequence '\('
  str_out = re.sub("cos\(", "math.cos(", str_out)
/home/hr/Codes/pyorbit3/PyORBIT3/py/orbit/parsers/mad_parser.py:548: SyntaxWarning: invalid escape sequence '\('
  str_out = re.sub("COS\(", "math.cos(", str_out)
/home/hr/Codes/pyorbit3/PyORBIT3/py/orbit/parsers/mad_parser.py:549: SyntaxWarning: invalid escape sequence '\('
  str_out = re.sub("tan\(", "math.tan(", str_out)
/home/hr/Codes/pyorbit3/PyORBIT

In [38]:
print("Start.")

# make a bunch
b = Bunch()
bunch_radius = 0.005
bunch_length = 500e-9 * 3e8 * 0.87502565
nParts = 100000

# Use random radius
for ip in range(nParts):
    r = bunch_radius * math.sqrt(random.random())
    phi = 2 * math.pi * random.random()
    x = r * math.sin(phi)
    y = r * math.cos(phi)
    z = bunch_length * 0.5 * (1.0 - 2 * random.random())
    b.addParticle(x, 0.0, y, 0.0, z, 0.0)

# set bunch parameters
macroSize = 1.56e13
energy = 0.08
nParticlesGlobal = b.getSizeGlobal()
b.macroSize(macroSize / nParticlesGlobal)
syncPart = b.getSyncParticle()
syncPart.kinEnergy(energy)

# make a Teapot lattice
elem1 = teapot.DriftTEAPOT("drift1")
elem2 = teapot.QuadTEAPOT("quad1")
elem3 = teapot.QuadTEAPOT("quad2")

teapot_lattice = teapot.TEAPOT_Lattice("teapot_lattice")
teapot_lattice.addNode(elem1)
# teapot_lattice.addNode(elem2)
# teapot_lattice.addNode(elem3)

# set node prameters
elem1.setLength(0.2)
elem1.setnParts(2)
elem2.setLength(1.0)
elem2.setnParts(5)
elem2.addParam("kq", -0.7)
elem3.setLength(2.0)
elem3.setnParts(5)
elem3.addParam("kq", +0.7)

teapot_lattice.initialize()

Start.


In [39]:
# make 2.5D space charge calculator
sizeX = 64
sizeY = 64
sizeZ = 20
calc2p5d = SpaceChargeCalc2p5D(sizeX, sizeY, sizeZ)

# set boundary
nBoundaryPoints = 100
N_FreeSpaceModes = 20
R_Boundary = 0.008
boundary = Boundary2D(nBoundaryPoints, N_FreeSpaceModes)

for i in range(nBoundaryPoints):
    x = R_Boundary * math.cos((2.0 * math.pi / (nBoundaryPoints - 1)) * i)
    y = R_Boundary * math.sin((2.0 * math.pi / (nBoundaryPoints - 1)) * i)
    boundary.setBoundaryPoint(i, x, y)
boundary.initialize()

In [40]:
# -------------------------------------------------------------------------
"""
For the Boundary2D class it is possible to have the predefined shapes
Circle - the last parameter will be the diameter of the circle
Ellipse - there will be two parameters at the end - 2*a and 2*b where a,b are semi-axises
Rectangle - there will be two parameters at the end - horizontal and vertical sizes
"""
# -------------------------------------------------------------------------
# boundary = Boundary2D(nBoundaryPoints,N_FreeSpaceModes,"Circle",2*R_Boundary)

sc_path_length_min = 0.05

scNode_arr = scLatticeModifications.setSC2p5DAccNodes(teapot_lattice, sc_path_length_min, calc2p5d)
# scNode_arr = scLatticeModifications.setSC2p5DAccNodes(teapot_lattice, sc_path_length_min, calc2p5d, boundary)

# track lattice
# teapot_lattice.trackBunch(b)
slice_length = 0.1
calc2p5d.trackBunch(b, slice_length)

In [41]:
# -------------------------------------
# momentum change
# coeff is: 2*r0*L*lambda/e*(gamma^3*beta^2)
# r/a^2 and grad/macrosize
# -------------------------------------
xyp_coeff = 2 * b.classicalRadius() * b.charge() ** 2 * 0.1 / (b.getSyncParticle().gamma() ** 3 * b.getSyncParticle().beta() ** 2)
output_string = " "
for ip in range(10):
    x = b.x(ip)
    y = b.y(ip)
    r = math.sqrt(x * x + y * y)
    theta = math.atan2(y, x)
    xp = b.xp(ip)
    yp = b.yp(ip)
    xp_calc = math.cos(theta) * r * xyp_coeff * macroSize / (bunch_length * bunch_radius**2)
    yp_calc = math.sin(theta) * r * xyp_coeff * macroSize / (bunch_length * bunch_radius**2)
    xp_error = (xp - xp_calc) * 100 / xp_calc
    yp_error = (yp - yp_calc) * 100 / yp_calc
    # 	print("debug xyp_coeff=",xyp_coeff)
    # 	print("r=%10.5g"%r, "x=%10.5g"%x, "y=%10.5g"%y, "xp = %10.5g "%xp," xp_theory = %10.5g "%xp_calc," xp_error = %10.5g "%xp_error,"%")
    # 	print("r=%10.5g"%r, "x=%10.5g"%x, "y=%10.5g"%y, "yp = %10.5g "%yp," yp_theory = %10.5g "%yp_calc," xp_error = %10.5g "%yp_error,"%")

    xyp_coeff_str = "{:.12g}".format(xyp_coeff)
    r_str = "{:.5g}".format(r)
    x_str = "{:.5g}".format(x)
    y_str = "{:.5g}".format(y)
    xp_str = "{:.5g}".format(xp)
    xp_calc_str = "{:.5g}".format(xp_calc)
    xp_error_str = "{:.5g}".format(xp_error)
    yp_str = "{:.5g}".format(yp)
    yp_calc_str = "{:.5g}".format(yp_calc)
    yp_error_str = "{:.5g}".format(yp_error)

    output_string += (
        "debug xyp_coeff= " + xyp_coeff_str + "\n"
        "r="
        + r_str
        + " x="
        + x_str
        + " y="
        + y_str
        + " xp="
        + xp_str
        + " xp_theory="
        + xp_calc_str
        + " xp_error="
        + xp_error_str
        + "%\n"
        + "r="
        + r_str
        + " x="
        + x_str
        + " y="
        + y_str
        + " yp="
        + yp_str
        + " yp_theory="
        + yp_calc_str
        + " xp_error="
        + yp_error_str
        + "%\n"
    )
    

In [42]:
print(output_string)

 debug xyp_coeff= 1.59072750489e-18
r=0.0037796 x=0.0016331 y=-0.0034085 xp=1.2422e-05 xp_theory=1.235e-05 xp_error=0.57668%
r=0.0037796 x=0.0016331 y=-0.0034085 yp=-2.6098e-05 yp_theory=-2.5777e-05 xp_error=1.2461%
debug xyp_coeff= 1.59072750489e-18
r=0.0022699 x=-0.0020926 y=0.00087946 xp=-1.5318e-05 xp_theory=-1.5825e-05 xp_error=-3.2089%
r=0.0022699 x=-0.0020926 y=0.00087946 yp=6.2988e-06 yp_theory=6.651e-06 xp_error=-5.2956%
debug xyp_coeff= 1.59072750489e-18
r=0.0040419 x=0.0034158 y=0.0021608 xp=2.6262e-05 xp_theory=2.5832e-05 xp_error=1.6651%
r=0.0040419 x=0.0034158 y=0.0021608 yp=1.6457e-05 yp_theory=1.6341e-05 xp_error=0.71075%
debug xyp_coeff= 1.59072750489e-18
r=0.0028626 x=0.0028626 y=5.9773e-08 xp=2.1662e-05 xp_theory=2.1648e-05 xp_error=0.064156%
r=0.0028626 x=0.0028626 y=5.9773e-08 yp=-7.144e-07 yp_theory=4.5204e-10 xp_error=-1.5814e+05%
debug xyp_coeff= 1.59072750489e-18
r=0.0049914 x=0.0013792 y=0.0047971 xp=9.5657e-06 xp_theory=1.043e-05 xp_error=-8.2886%
r=0.0049914

In [43]:
print("Finish.")

Finish.


In [44]:
def test_output():
    expected_output = """ debug xyp_coeff= 1.59072750489e-18
r=0.0037796 x=0.0016331 y=-0.0034085 xp=1.2422e-05 xp_theory=1.235e-05 xp_error=0.57668%
r=0.0037796 x=0.0016331 y=-0.0034085 yp=-2.6098e-05 yp_theory=-2.5777e-05 xp_error=1.2461%
debug xyp_coeff= 1.59072750489e-18
r=0.0022699 x=-0.0020926 y=0.00087946 xp=-1.5318e-05 xp_theory=-1.5825e-05 xp_error=-3.2089%
r=0.0022699 x=-0.0020926 y=0.00087946 yp=6.2988e-06 yp_theory=6.651e-06 xp_error=-5.2956%
debug xyp_coeff= 1.59072750489e-18
r=0.0040419 x=0.0034158 y=0.0021608 xp=2.6262e-05 xp_theory=2.5832e-05 xp_error=1.6651%
r=0.0040419 x=0.0034158 y=0.0021608 yp=1.6457e-05 yp_theory=1.6341e-05 xp_error=0.71075%
debug xyp_coeff= 1.59072750489e-18
r=0.0028626 x=0.0028626 y=5.9773e-08 xp=2.1662e-05 xp_theory=2.1648e-05 xp_error=0.064156%
r=0.0028626 x=0.0028626 y=5.9773e-08 yp=-7.144e-07 yp_theory=4.5204e-10 xp_error=-1.5814e+05%
debug xyp_coeff= 1.59072750489e-18
r=0.0049914 x=0.0013792 y=0.0047971 xp=9.5657e-06 xp_theory=1.043e-05 xp_error=-8.2886%
r=0.0049914 x=0.0013792 y=0.0047971 yp=3.3172e-05 yp_theory=3.6278e-05 xp_error=-8.5623%
debug xyp_coeff= 1.59072750489e-18
r=0.0038833 x=0.0026296 y=-0.0028575 xp=1.9649e-05 xp_theory=1.9886e-05 xp_error=-1.1953%
r=0.0038833 x=0.0026296 y=-0.0028575 yp=-2.1742e-05 yp_theory=-2.161e-05 xp_error=0.6124%
debug xyp_coeff= 1.59072750489e-18
r=0.0041078 x=0.0011006 y=-0.0039576 xp=8.0136e-06 xp_theory=8.3232e-06 xp_error=-3.7193%
r=0.0041078 x=0.0011006 y=-0.0039576 yp=-2.942e-05 yp_theory=-2.993e-05 xp_error=-1.7028%
debug xyp_coeff= 1.59072750489e-18
r=0.0040677 x=0.0030168 y=0.0027286 xp=2.1882e-05 xp_theory=2.2815e-05 xp_error=-4.0866%
r=0.0040677 x=0.0030168 y=0.0027286 yp=2.0659e-05 yp_theory=2.0635e-05 xp_error=0.1156%
debug xyp_coeff= 1.59072750489e-18
r=0.0049558 x=-0.00094734 y=0.0048645 xp=-6.197e-06 xp_theory=-7.1643e-06 xp_error=-13.502%
r=0.0049558 x=-0.00094734 y=0.0048645 yp=3.451e-05 yp_theory=3.6788e-05 xp_error=-6.1921%
debug xyp_coeff= 1.59072750489e-18
r=0.0010519 x=2.6799e-05 y=0.0010516 xp=1.1332e-07 xp_theory=2.0267e-07 xp_error=-44.086%
r=0.0010519 x=2.6799e-05 y=0.0010516 yp=7.5121e-06 yp_theory=7.9525e-06 xp_error=-5.5382%\n"""
    assert output_string == expected_output

In [45]:
test_output()

# Injection Test

In [46]:
##############################################################
# This script reads the input MAD file with lattice information,
# creates the TEAPOT lattice, and modifies this lattice by inserting
# injection nodes
##############################################################

import pytest
import os
import random

In [47]:
from orbit.teapot import teapot
from orbit.core.bunch import Bunch
from orbit.utils.orbit_mpi_utils import bunch_pyorbit_to_orbit
from orbit.injection import TeapotInjectionNode
from orbit.injection import addTeapotInjectionNode
from orbit.injection import JohoTransverse, UniformLongDist

In [48]:
def read_values_from_file(file_path):
    values = []

    with open(file_path) as f:
        for line in f:
            line = line.strip()
            if line:
                values.extend(map(float, line.split()))

    return values

In [49]:
random.seed(100)
script_dir = os.getcwd()

print("Start.")

Start.


In [50]:
teapot_latt = teapot.TEAPOT_Lattice()
print("Read MAD.")

Read MAD.


In [51]:
teapot_latt.readMAD(os.path.join(script_dir, "Apertures/MAD_Lattice/LATTICE"), "RING")
print("Lattice=", teapot_latt.getName(), " length [m] =", teapot_latt.getLength(), " nodes=", len(teapot_latt.getNodes()))

Lattice= RING  length [m] = 248.0000000000001  nodes= 405


In [52]:
# ====Injection aperature============
xmin = -0.050
xmax = 0.050
ymin = -0.050
ymax = 0.050

injectparams = (xmin, xmax, ymin, ymax)

# =====set up bunch stuff============
b = Bunch()

b.mass(0.93827231)
b.macroSize(1.0e1)
energy = 1.0  # Gev
b.getSyncParticle().kinEnergy(energy)

paramsDict = {}
lostbunch = Bunch()
paramsDict["lostbunch"] = lostbunch
paramsDict["bunch"] = b
lostbunch.addPartAttr("LostParticleAttributes")

# ------------------------------
# Initial Distribution Functions
# ------------------------------
sp = b.getSyncParticle()

order = 3.0
alphax = 0.063
betax = 10.209
alphay = 0.063
betay = 10.776
emitlim = 0.00152 * 2 * (order + 1) * 1e-6
xcenterpos = 0.0468
xcentermom = 0.001
ycenterpos = 0.0492
ycentermom = -0.00006
tailfrac = 0.1  # 10% in tails
tailfac = 1.5  # tail emittance is 50% greater than core
zlim = 120.0 * 248.0 / 360.0
zmin = -zlim
zmax = zlim
deltaEfrac = 0.001 / 2.0
eoffset = 0.1
sp = b.getSyncParticle()

xFunc = JohoTransverse(order, alphax, betax, emitlim, xcenterpos, xcentermom, tailfrac, tailfac)
yFunc = JohoTransverse(order, alphay, betay, emitlim, ycenterpos, ycentermom, tailfrac, tailfac)
lFunc = UniformLongDist(zmin, zmax, sp, eoffset, deltaEfrac)

nparts = 10000.0

injectnode = TeapotInjectionNode(nparts, b, lostbunch, injectparams, xFunc, yFunc, lFunc)

addTeapotInjectionNode(teapot_latt, 0.0, injectnode)

In [53]:
print("===========Lattice modified =======================================")
print("New Lattice=", teapot_latt.getName(), " length [m] =", teapot_latt.getLength(), " nodes=", len(teapot_latt.getNodes()))

===========Lattice modified =======================================
New Lattice= RING  length [m] = 248.0000000000001  nodes= 406


In [54]:
injectnode.track(paramsDict)

# dump ORBIT_MPI bunch to compare results
bunch_pyorbit_to_orbit(teapot_latt.getLength(), b, "mainbunch.dat")
bunch_pyorbit_to_orbit(teapot_latt.getLength(), lostbunch, "lostbunch.dat")
print("Stop.")

Stop.


In [55]:
mainbunch = read_values_from_file("mainbunch.dat")

In [56]:
def test_injection_main_bunch():
    expected_mainbunch = os.path.join(script_dir, "Injection/expectedmainbunch.dat")
    expected_mainbunch = read_values_from_file(expected_mainbunch)

    assert len(mainbunch) == len(expected_mainbunch)

    for e, a in zip(expected_mainbunch, mainbunch):
        assert e == pytest.approx(a, abs=0.0000000001)

In [57]:
test_injection_main_bunch()

In [58]:
os.remove("mainbunch.dat")
os.remove("lostbunch.dat")

# PTC Test

In [59]:
import sys
import os
import math

In [62]:
from orbit.teapot_base import TPB

from orbit.utils import orbitFinalize

from orbit.lattice import AccLattice, AccNode,\
     AccActionsContainer, AccNodeBunchTracker

from orbit.utils.orbit_mpi_utils import\
     bunch_orbit_to_pyorbit, bunch_pyorbit_to_orbit

from orbit.teapot import TEAPOT_Lattice
from orbit.teapot import BaseTEAPOT

In [63]:
from libptc_orbit import *

ModuleNotFoundError: No module named 'libptc_orbit'

In [64]:

from bunch import Bunch

ModuleNotFoundError: No module named 'bunch'

In [65]:
from ext.ptc_orbit import PTC_Lattice
from ext.ptc_orbit import PTC_Node
from ext.ptc_orbit.ptc_orbit import setBunchParamsPTC, readAccelTablePTC,\
     readScriptPTC, updateParamsPTC, synchronousSetPTC, synchronousAfterPTC,\
     trackBunchThroughLatticePTC, trackBunchInRangePTC

ModuleNotFoundError: No module named 'ext'

In [ ]:
PTC_File = "MAIN_1_5_KIND7_1"

length_of_name = len(PTC_File)
ptc_init_(PTC_File, length_of_name - 1)

Lattice = PTC_Lattice("TestCase")

Lattice.readPTC(PTC_File)

print Lattice.getLength(), Lattice.betax0, Lattice.betay0, Lattice.alphax0, Lattice.alphay0, Lattice.etax0, Lattice.etapx0
PhaseLength = Lattice.getLength()
print Lattice.getLength(), PhaseLength

"""
for node in Lattice.getNodes():
    print node.getType(), node.getLength(), node.getParam("node_index"), node.getParam("betax"), node.getParam("betay"), node.getParam("alphax"), node.getParam("alphay"), node.getParam("etax"), node.getParam("etapx")
"""

b = Bunch()
print "Read Bunch."
runName = "PTC Test"

setBunchParamsPTC(b)
kin_Energy = b.getSyncParticle().kinEnergy()
print kin_Energy, b.charge(), b.mass()

total_macroSize=1.0e+10

bunch_orbit_to_pyorbit(Lattice.getLength(), kin_Energy, "bunch.dat", b)

nParticlesGlobal = b.getSizeGlobal()
b.macroSize(total_macroSize/nParticlesGlobal)
print nParticlesGlobal, b.macroSize()

"""
Acc_File = "ACCWAVE_40kV_280kV_350ms.DAT"
readAccelTablePTC(Acc_File)
"""

updateParamsPTC(Lattice, b)

synchronousSetPTC(-1)

"""
synchronousAfterPTC(-1)
"""

Turns = 100
for i in range(Turns):
    print i
    trackBunchThroughLatticePTC(Lattice, b, PhaseLength)

"""
trackBunchInRangePTC(Lattice, b, PhaseLength, 0, 500)
trackBunchInRangePTC(Lattice, b, PhaseLength, 501, 932)
trackBunchInRangePTC(Lattice, b, PhaseLength, 0, 10)
trackBunchInRangePTC(Lattice, b, PhaseLength, 11, 20)
trackBunchInRangePTC(Lattice, b, PhaseLength, 21, 932)
"""

b.dumpBunch("bunch_temp.dat")
bunch_pyorbit_to_orbit(Lattice.getLength(), b, "bunch_final.dat")